In [1]:
# prompt: from google.colab improt drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 67.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ------------------------------------ 164.2/164.2 MB 146.5 kB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 224.8 kB/s eta 0:00:00
     ---------------------------------------- 2.3/2.3 MB 200.9 kB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\my pc\\anaconda3\\envs\\saeed\\lib\\site-packages\\~orch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.



In [ ]:
import os
import torch
from transformers import BertTokenizer, BertModel, T5Tokenizer, T5EncoderModel
from Bio import SeqIO
import numpy as np
from tqdm import tqdm
import pandas as pd  # Added for saving CSV




def chunk_sequence(seq, chunk_size=512, overlap=0):
    """Split long sequence into chunks of max length (optionally with overlap)."""
    step = chunk_size - overlap
    return [seq[i:i + chunk_size] for i in range(0, len(seq), step)]

def tokenize_sequence(seq, tokenizer, model_type="bert"):
    seq = ' '.join(list(seq))  # space-separate amino acids
    return tokenizer(seq, return_tensors="pt", add_special_tokens=True, truncation=True).to(device)

def extract_embeddings(sequences, model, tokenizer, model_type="bert", method="average"):
    all_embeddings = []
    sequence_ids = []

    for seq_id, seq in tqdm(sequences):
        # Split into 512-length chunks
        chunks = chunk_sequence(seq, chunk_size=512)
        chunk_embeddings = []

        for chunk in chunks:
            inputs = tokenize_sequence(chunk, tokenizer, model_type)
            with torch.no_grad():
                outputs = model(**inputs)
                last_hidden = outputs.last_hidden_state.squeeze(0)

                if method == "cls":
                    emb = last_hidden[0]
                elif method == "average":
                    emb = last_hidden.mean(dim=0)
                elif method == "center":
                    mid = (last_hidden.shape[0] - 1) // 2
                    emb = last_hidden[mid]
                else:
                    raise ValueError("Invalid method: choose from 'cls', 'average', 'center'")

                chunk_embeddings.append(emb.cpu().numpy())

        # Average all chunk embeddings for final representation
        final_embedding = np.mean(chunk_embeddings, axis=0)
        all_embeddings.append(final_embedding)
        sequence_ids.append(seq_id)

    return sequence_ids, np.array(all_embeddings)



# === Set your values here ===
path = 'D:/A Max-project/10-NTxPred/'  # Path to your dataset
print("Current Working Directory:", os.getcwd())
fasta_file = path + 'Features_extraction/datasets/proteins/independent_dataset_protein.fasta'
output_csv = path + 'features/proteins/IND_bert_features11.csv'        # Output CSV file
model_name = "bert-base"                       # Options: 'bert-base', 'bert-prot', 'bert-tape'
method = "average"                             # Options: 'cls', 'average', 'center'
# ============================
#cross_val_dataset_peptides independent_dataset_peptides
#cross_val_dataset_protein   independent_dataset_protein
#cross_val_dataset_combined  independent_dataset_combined
def main():
    sequences = load_fasta(fasta_file)
    tokenizer, model, model_type = load_model_and_tokenizer(model_name)
    sequence_ids, embeddings = extract_embeddings(sequences, model, tokenizer, model_type, method)

    # Convert to DataFrame and save as CSV
    df = pd.DataFrame(embeddings)
    df.insert(0, "ID", sequence_ids)
    df.to_csv(output_csv, index=False)
    print(f"Saved embeddings to {output_csv}")

if __name__ == "__main__":
    main()


Current Working Directory: d:\A Max-project\10-NTxPred\Features_extraction\LLM


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


NameError: name 'device' is not defined